# Setting up inputs for the gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-10  
  
In this notebook, we will prepare all necessary inputs need to run the gridded version of DBPM. Note that this step needs to be completed only once for each model run.  

## Loading relevant libraries

In [1]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
import xarray as xr
import json
import pandas as pd
import numpy as np
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33645 instead
  warnings.warn(


## Defining output folder

In [2]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/'
out_folder = os.path.join(base_folder, 'gridded_params')
os.makedirs(out_folder, exist_ok = True)

## Transforming DBPM parameters to Python-friendly format
These parameters are the outputs of the `sizeparam` function for R. They were calculated and stored in the script ([04_calculating_dbpm_fishing_params](new_workflow/04_calculating_dbpm_fishing_params.R)) as a `json` file.

In [3]:
#Loading gridded parameters
gridded_params = json.load(open(
    os.path.join(out_folder, 'dbpm_gridded_size_params_fao-58.json')))

#Transforming them to Python friendly format
gridded_python = uf.gridded_param_python(gridded_params)

### Adding useful entries to gridded parameters
These variables are used multiple times within the `gridded_size_model` script, so they will be added for convenience.

In [4]:
# Size classes
log10_size_bins = np.array(gridded_python['log10_size_bins'])
# Minimum (log10) size class for predators
log10_ind_min_pred_size = log10_size_bins[gridded_python['ind_min_pred_size']]
ind_min_detritivore_size = gridded_python['ind_min_detritivore_size']
log10_ind_min_detritivore_size = log10_size_bins[ind_min_detritivore_size]
log10_ind_min_fish_pred = log10_size_bins[gridded_python['ind_min_fish_pred']]
log10_ind_min_fish_det = log10_size_bins[gridded_python['ind_min_fish_det']]

#Adding new variables to gridded parameters
gridded_python['log10_ind_min_pred_size'] = log10_ind_min_pred_size
gridded_python['log10_ind_min_detritivore_size'] = log10_ind_min_detritivore_size

### Saving gridded parameters

In [5]:
out_gridded = os.path.join(out_folder, 'dbpm_gridded_size_params_fao-58_python.json')

#Save to disk
with open(out_gridded, 'w') as outfile:
    json.dump(gridded_python, outfile)

## Loading gridded input data
Gridded data are needed both as direct input to DBPM and to calculate params, such as habitat preference, pelagic predator and benthic detritivore size spectrum, among other things.

In [5]:
depth = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_deptho_15arcmin_fao-58_fixed.zarr/'))['deptho']

In [6]:
#Loading obsclime data and fixing time
obs_clim_data  = xr.open_zarr(
    os.path.join(base_folder, 'gridded',
                 'gfdl-mom6-cobalt2_obsclim_intercept_15arcmin_fao-58_monthly_1961_2010.zarr/'))
obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

#Concatenating with spinup data
int_phy_zoo = xr.concat(
    [xr.open_zarr(
        os.path.join(base_folder, 'gridded',
                     'gfdl-mom6-cobalt2_spinup_intercept_15arcmin_fao-58_monthly_1841_1960.zarr/')),
     obs_clim_data], dim = 'time')['intercept']

/jobfs/132307763.gadi-pbs/ipykernel_298168/3836049655.py:5: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),


In [60]:
#Loading obsclime data and fixing time
obs_clim_data  = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_slope_15arcmin_fao-58_monthly_1961_2010.zarr/'))
obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

slope_phy_zoo = xr.concat(
    [xr.open_zarr(os.path.join(base_folder, 'gridded', 
                               'gfdl-mom6-cobalt2_spinup_slope_15arcmin_fao-58_monthly_1841_1960.zarr/')),
    obs_clim_data], dim = 'time')['slope']

/jobfs/132222511.gadi-pbs/ipykernel_1558644/4148534442.py:5: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),


In [61]:
#Loading obsclime data and fixing time
obs_clim_data  = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_tos_15arcmin_fao-58_monthly_1961_2010.zarr/'))
obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')

sea_surf_temp = xr.concat(
    [xr.open_zarr(os.path.join(base_folder, 'gridded',
                               'gfdl-mom6-cobalt2_spinup_tos_15arcmin_fao-58_monthly_1841_1960.zarr')),
     obs_clim_data], dim = 'time')['tos']

/jobfs/132222511.gadi-pbs/ipykernel_1558644/3751439967.py:5: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),


In [62]:
#Loading obsclime data and fixing time
obs_clim_data  = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_tob_15arcmin_fao-58_monthly_1961_2010.zarr/'))
obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),
                                         dtype = 'datetime64[ns]')
sea_floor_temp = xr.concat(
    [xr.open_zarr(os.path.join(base_folder, 'gridded',
                               'gfdl-mom6-cobalt2_spinup_tob_15arcmin_fao-58_monthly_1841_1960.zarr')),
     obs_clim_data], dim = 'time')['tob']

/jobfs/132222511.gadi-pbs/ipykernel_1558644/610008515.py:5: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  obs_clim_data['time'] = pd.DatetimeIndex(obs_clim_data.indexes['time'].to_datetimeindex(),


### Creating time variable that contains a timestep for initialisation of DBPM

In [8]:
#Adding new 'time_init' input that includes timestep for initialisation of DBPM
time_init = (np.append((int_phy_zoo.time.min().values-np.timedelta64(1, 'm')),
                       int_phy_zoo.time.values))

## Creating gridded variables needed for DBPM run
We will create data array versions of the gridded inputs.

### DBPM size classes

In [9]:
# Log10 size of individuals found in the model
log10_size_bins_mat = xr.DataArray(data = log10_size_bins,
                                   dims = ['size_class'], 
                                   coords = {'size_class': log10_size_bins})

# size of individuals found in the model
size_bin_vals = 10**log10_size_bins_mat

### Fishing effort

In [58]:
effort = (xr.DataArray(gridded_python['effort'], dims = 'time', 
                       coords = {'time': int_phy_zoo.time}).
          expand_dims({'lat': depth.lat, 'lon': depth.lon}).
          transpose('time', 'lat', 'lon'))

### Setting habitat preferences
Predator coupling to benthic prey, which is depth dependent, 0.75 above 500 m, 0.5 between 500 and 1800 m, and 0 below 1800 m. Predator coupling to pelagic prey is equal to `1-(benthic coupling)`.  
  
These values were suggested by Clive Trueman based on stable isotope work, and proportion of biomass. Rockall Trough studies.

In [11]:
pref_benthos = (0.8*np.exp((-1/250*depth)))
pref_pelagic = (1-pref_benthos)

### Initiliasing values for pelagic predators and benthic detritivores

In [12]:
init_pred = (xr.DataArray(data = gridded_python['init_pred'], 
                          dims = ['size_class'], 
                          coords = {'size_class': log10_size_bins}).
             expand_dims({'lat': depth.lat, 'lon': depth.lon}))

init_detritivores = (xr.DataArray(data = gridded_python['init_detritivores'], 
                                  dims = ['size_class'], 
                                  coords = {'size_class': log10_size_bins}).
                     expand_dims({'lat': depth.lat, 'lon': depth.lon}))

### Calculating constant growth and mortality

In [13]:
pred_prey_mat = uf.pred_prey_matrix(log10_size_bins)

constant_growth = xr.DataArray(uf.gphi_f(pred_prey_mat, 
                                         gridded_python['log10_pred_prey_ratio'],
                                         gridded_python['log_prey_pref']),
                               dims = ['size_class', 'sc'])

constant_mortality = xr.DataArray(uf.mphi_f(-pred_prey_mat, 
                                            gridded_python['log10_pred_prey_ratio'],
                                            gridded_python['log_prey_pref'],
                                            gridded_python['metabolic_req_pred']),
                                  dims = ['size_class', 'sc'])

### Building metabolic requirements lookup table 
Metabolic requirements are estimated per size class

In [14]:
met_req_log10_size_bins = uf.expax_f(log10_size_bins_mat, 
                                     gridded_python['metabolic_req_pred'])

In [15]:
consume_pelagic = (pref_pelagic*
                   gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)
consume_benthos = (pref_benthos*
                   gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)

### Creating a gridded time series of intercept of plankton size spectrum 
The size spectrum was estimated by the `GetPPIntSlope` function using GFDL-MOM6-COBALT2 outputs in script ([01_processing_dbpm_global_inputs](new_workflow/01_processing_dbpm_global_inputs.ipynb)).

In [16]:
ui0 = 10**int_phy_zoo

### Predator and detritivore biomass, plus detritus
Values are derived from plankton-zooplankton size spectrum, slope and size class.

In [17]:
# Calculating predator biomass
pred_start = ui0*(10**(slope_phy_zoo*log10_size_bins_mat))
# Keep values outside the predator size classes, otherwise change to zero (0)
pred_start = xr.where(pred_start.size_class <= log10_ind_min_pred_size,
                     pred_start, 0)
# Copy first time step predator values as 'initialisation' timestep
pred_start = xr.concat([pred_start.isel(time = 0), pred_start], dim = 'time')
# Update time labels
pred_start['time'] = time_init
# Assigning initial values (time = 0) for predators from gridded parameters
predators = xr.where((pred_start.time == pred_start.time.min()) & 
                      (pred_start.size_class >= log10_ind_min_pred_size),
                     init_pred, pred_start)

In [49]:
# Assigning initial values (time = 0) for detritivores from gridded parameters
detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size),
                        init_detritivores, 0)
# Adding time dimension
detritivores = detritivores.expand_dims({'time': [min(time_init)]})

# Initialising detritus
detritus = xr.zeros_like(detritivores.isel(size_class = 0).drop_vars('size_class'))
# Assigning initial values (time = 0) for detritus from gridded parameters
detritus = xr.where(detritus == 0, gridded_python['init_detritus'], detritus)

### Note
If no initial values for `predators` and `detritivores` are included in the gridded parameters, use the following code instead:  
  
```
predators = xr.where((pred_start.time == pred_start.time.min()) &
                       (pred_start.size_class >= log10_ind_min_pred_size) &
                       (pred_start.size_class < log10_size_bins[120]),
                       init_pred, pred_start)

detritivores = xr.where((init_detritivores.size_class >= log10_ind_min_detritivore_size) &
                         (init_detritivores.size_class < log10_size_bins[120]), 
                         init_detritivores, detri_init)
```

### Fishing mortality
Here `fishing_mort_pred` and `fishing_mort_det` are fixed catchability terms for `predators` and `detritivores`. Fishing mortality is estimated only for individuals within fished sizes (`ind_min_fish_pred` or `ind_min_fish_det`).

In [53]:
# Using fish_mort_pred as a base for fishing mortality for predators
fish_mort_pred = xr.DataArray(
    np.repeat(gridded_python['fish_mort_pred'], 
              np.prod(consume_pelagic.shape)).reshape(consume_pelagic.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_pelagic.lat, 'lon': consume_pelagic.lon,
             'size_class': consume_pelagic.size_class})

# Changing mortality to zero outside the predator size fished
fishing_mort_pred = xr.where((fish_mort_pred.size_class >= log10_ind_min_fish_pred) &
                          (fish_mort_pred.size_class < 
                           fish_mort_pred.size_class.max()),
                          fish_mort_pred, 0)

In [55]:
# Using fish_mort_det as a base for fishing mortality for detritivores
fish_mort_det = xr.DataArray(
    np.repeat(gridded_python['fish_mort_detritivore'], 
              np.prod(consume_benthos.shape)).reshape(consume_benthos.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_benthos.lat, 'lon': consume_benthos.lon,
             'size_class': consume_benthos.size_class})

# Changing mortality to zero outside the detritivore size fished
fishing_mort_det = xr.where((fish_mort_det.size_class >= log10_ind_min_fish_det) &
                          (fish_mort_det.size_class < 
                           fish_mort_det.size_class.max()),
                          fish_mort_det, 0)

### Temperature effect on pelagic and benthic organisms

In [57]:
pel_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_surf_temp+273)))

ben_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_floor_temp+273)))

In [10]:
# int_phy_zoo = xr.open_zarr(
#     os.path.join(base_folder, 'gridded',
#                  'gfdl-mom6-cobalt2_spinup_intercept_15arcmin_fao-58_monthly_1841_1960.zarr/'))['intercept']

# slope_phy_zoo = xr.open_zarr(
#     os.path.join(base_folder, 'gridded', 
#                  'gfdl-mom6-cobalt2_spinup_slope_15arcmin_fao-58_monthly_1841_1960.zarr/'))['slope']

# sea_surf_temp = xr.open_zarr(
#     os.path.join(base_folder, 'gridded',
#                  'gfdl-mom6-cobalt2_spinup_tos_15arcmin_fao-58_monthly_1841_1960.zarr'))['tos']

# sea_floor_temp = xr.open_zarr(
#     os.path.join(base_folder, 'gridded',
#                  'gfdl-mom6-cobalt2_spinup_tob_15arcmin_fao-58_monthly_1841_1960.zarr'))['tob']



# yr = int_phy_zoo.time.dt.year
# dec = ((yr - yr[0])/10).astype(int)

# int_phy_zoo = int_phy_zoo.assign_coords({'dec': dec})

# for i, da in int_phy_zoo.groupby('dec'):

#     print(da.time.dt.year)

effort = (xr.DataArray(gridded_python['effort'][:len(int_phy_zoo.time)], 
                       dims = 'time', coords = {'time': int_phy_zoo.time}).
          expand_dims({'lat': depth.lat, 'lon': depth.lon}).
          transpose('time', 'lat', 'lon'))

### Note
If `temp_effect` is not needed, then simply assign `1` to `pel_tempeffect` and `ben_tempeffect`:  
  
```
pel_tempeffect = 1
ben_tempeffect = 1
```

## Saving gridded inputs
We will now save all the gridded inputs needed to run the DBPM model.

In [34]:
log10_size_bins_mat.name = 'size_bins'
log10_size_bins_mat.to_zarr('outputs/log10_size_bins_matrix.zarr/',
                            consolidated = True, mode = 'w')

In [138]:
effort.name = 'effort'
effort_out = effort.chunk({'time': '50MB'})
effort_out.to_zarr(os.path.join(out_folder, 'effort_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                   consolidated = True, mode = 'w')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 791.90 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [140]:
pref_benthos.name = 'pref_benthos'
pref_benthos.to_zarr(os.path.join(out_folder, 
                                  'pref-benthos_15arcmin_fao-58_fixed.zarr/'),
                     consolidated = True, mode = 'w')
pref_pelagic.name = 'pref_pelagic'
pref_pelagic.to_zarr(os.path.join(out_folder, 
                                  'pref-pelagic_15arcmin_fao-58_fixed.zarr/'), 
                     consolidated = True, mode = 'w')

In [142]:
constant_growth.name = 'constant_growth'
constant_growth.to_zarr(os.path.join(out_folder,
                                     'const-growth_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

constant_mortality.name = 'constant_mortality'
constant_mortality.to_zarr(os.path.join(out_folder,
                                     'const-mort_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [145]:
consume_pelagic.name = 'consume_pelagic'
consume_pelagic.to_zarr(os.path.join(out_folder,
                                     'consume-pelagic_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

consume_benthos.name = 'consume_benthos'
consume_benthos.to_zarr(os.path.join(out_folder,
                                     'consume-benthos_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [ ]:
predators.name = 'predators'
predators_out = predators.chunk({'time': '200MB', 'lat': 106, 'lon': 480})
predators_out.to_zarr(
    os.path.join(out_folder, 'predators_15arcmin_fao-58_monthly_1961_2010.zarr/'),
    consolidated = True, mode = 'w')

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.07/lib/python3.10/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 72.00 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2025-01-13 16:53:04,018 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.33 GiB -- Worker memory limit: 4.47 GiB
2025-01-13 16:53:05,944 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.da

In [ ]:
detritivores.name = 'detritivores'
detritivores = detritivores.chunk({'time': 13, 'size_class': 181,
                                   'lat': 23, 'lon': '50MB'})
detritivores.to_zarr(os.path.join(out_folder, 
                                  'detritivores_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                     consolidated = True, mode = 'w')

detritus.name = 'detritus'
detritus = detritus.chunk({'time': '50MB', 'lat': 23,
                           'lon': '50MB'})
detritus.to_zarr(os.path.join(out_folder, 
                              'detritus_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                 consolidated = True, mode = 'w')

In [36]:
fishing_mort_pred.name = 'fish_mort_pred'
fishing_mort_pred.to_zarr(os.path.join(out_folder, 
                                       'fish-mort-pred_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')

fishing_mort_det.name = 'fish_mort_det'
fishing_mort_det.to_zarr(os.path.join(out_folder, 
                                      'fish-mort-det_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                         consolidated = True, mode = 'w')

In [67]:
pel_tempeffect.name = 'pel_temp_eff' 
pel_tempeffect = pel_tempeffect.chunk({'lat': 23})
pel_tempeffect.to_zarr(os.path.join(out_folder, 
                                       'pel-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')

ben_tempeffect.name = 'ben_temp_eff'
ben_tempeffect = ben_tempeffect.chunk({'lat': 23})
ben_tempeffect.to_zarr(os.path.join(out_folder, 
                                       'ben-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')